# [RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

## [Overview](https://spark.apache.org/docs/latest/rdd-programming-guide.html#overview)

**EN**: 
At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures.
<br/>

**TR**:

Yüksek seviyede her Spark uygulaması bir ***driver program***' dan oluşur. ***driver program*** dediğimiz şey ise kullanıcının ana fonksiyonunu çalıştıran ve cluster üzerinde çeşitli paralel işlemler yapan yapıdır. [Spark applications run as independent sets of processes on a cluster, coordinated by the SparkContext object in your main program (called the driver program)]

> 6 makineli bir cluster var. Bu cluster' da bir tane master ve 5 tane slave makinesi var. Master, 5 slave makineyi de yöneten makinedir. Benim tahminim Spark' taki master makinede SparkContext oluşturulur yani ***driver program*** ve Spark 5 slave makine üzerinde paralel işlemler gerçekleştirir.

Spark' ın sağladığı **main absraction** **RDD** (*resilient distributed dataset*)' dir. RDD, bir cluster' daki node' lar boyunca dağılmış **element**' ler kümesidir. **Benim anladığım şu:** Elimizde 5 makineli bir cluster ve 3GB dosya olsun. Bu dosya, bu cluster' da dağıtık bir şekilde tutulur. Yani dosya parçalara ayrılır ve 5 bilgisayara da dağıtılır. Spark ise bu cluster' daki dağıtık veriyi RDD denen bir şeye dönüştürerek dosya üzerinde işlem yapabilir. Farklı makinelerdeki aynı dosyanın parçalarına -Hadoop' ta olduğu gibi- bütünleşikmiş gibi işlem yapılır.

RDD HDFS' deki bir dosyadan (ya da diğer Hadoop destekli dosya sistemlerinden) ya da **driver program**daki Scala collection(array)' undan transform edilerek oluşturulur. Kullanıcılar Spark' a bu RDD' nin RAM(in-memory)' de kalıcı olarak durmasını isteyebilirler ki bu durum paralel işlemlerde verimlilik getirir. **Daha türkçesi:** RDD' yi ya elle oluşturabilirsin ya da bir yerden dosya okuyarak oluşturabilirsin. Oluşturduktan sonra RAM' de kalmasını sağlayabilirsin ki bu sayede verimlilik sağlarsın(mesela o RDD' ye sürekli erişilme ihtiyacı varsa).

___

**EN:**

A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only “added” to, such as counters and sums.    

<br/>

**TR:**

Spark' ın **second abstraction**' ı paralel işlemlerde kullanılan ***shared variables***' dır. Spark' da farklı nodelar üzerinde birden fazla paralel task çalıştırılacak olsun. Spark her bir task için kullanılacak değişkenleri gerekli node' a iletir. Bu süreç Spark' ın default davranışıdır. Lakin bazen tasklar arasında ya da tasklar ile driver program arasında bazı değişkenlerin **share** edilmesi gerekir. İşte ***shared variables*** bu durumlarda kullanılır. İki tür ***shared variables*** vardır: tüm node' lardaki bellekte(sanırım Disk) bulunan bir değeri cache(RAM)' e alan broadcast, yalnızca eklenen(added) değişkenler olan accumulators.

> ***shared variables*** tiplerini anlamadım.


## [Linking with Spark](https://spark.apache.org/docs/latest/rdd-programming-guide.html#linking-with-spark)
Bu kısmı yazmadım.

## [Initializing Spark](https://spark.apache.org/docs/latest/rdd-programming-guide.html#initializing-spark)

EN:

The first thing a Spark program must do is to create a SparkContext object, which tells Spark how to access a cluster. To create a SparkContext you first need to build a SparkConf object that contains information about your application.

TR:

Bir Spark programının yapması gereken ilk şey `SparkContext` objesinden bir nesne üretmektir. `SparkContext` nesnesi Spark' a cluster' a nasıl erişmesi gerektiğini bildiren yapıdır. `SparkContext` oluşturmak için ise `SparkConf` objesi oluşturulmalıdır. `SparkConf` objesi oluşturduğunuz uygulama ile ilgili bilgileri tutan nesnedir.

----

EN:

Only one SparkContext should be active per JVM. You must stop() the active SparkContext before creating a new one.

TR:

`JVM` başına sadece bir tane `SparkContext` aktif edilebilir. [Benim anladığım bir makine de aynı anda tek Spark job' ı çalışabilir.] Yeni bir `SparkContext` oluşturmak için aktif olanı `stop()` etmeniz gerekmektedir.

------

```scala
val conf = new SparkConf().setAppName(appName).setMaster(master)
new SparkContext(conf)
```

EN:

The appName parameter is a name for your application to show on the cluster UI. master is a Spark, Mesos or YARN cluster URL, or a special “local” string to run in local mode. In practice, when running on a cluster, you will not want to hardcode master in the program, but rather launch the application with spark-submit and receive it there. However, for local testing and unit tests, you can pass “local” to run Spark in-process.

TR:

`appName` parametresi uygulamanız için Cluster UI' da gözükecek isimdir. [Cluster (birden fazla makineyi bağlayarak oluşturulan yapı) üzerinde çalıştırmadım o sebeple bilmiyor lakin Spark' ı local olarak çalıştırdığında bir UI linki veriyor: 192.XXX.XXX.XXX gibi bundan bahsediyor sanırım Cluster UI diye]. `master` ifadesi ise ya [Spark, Mesos or YARN cluster URL](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls) olmalı ya da Spark' ın local mode' u olan "local" ibaresi olmalı. Bir cluster üzerinde çalıştığınızda siz master parametresine sabit bir değer atamak istemeyeceksiniz bunun yerine spark-submit kullanacaksınız. Bla Bla Bla.


## [Using the Shell](https://spark.apache.org/docs/latest/rdd-programming-guide.html#using-the-shell)

Bakmadım buna da. Gereksiz geldi bana.

## [Resilient Distributed Datasets (RDDs)](https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds)

Spark ***Resilient Distributed Datasets*** kavramı etrafında döner. RDD, paralel olarak çalıştırılabilen, hata toleranslı elementler koleksiyonudur (**fault-tolerant** collection of elements).

RDD iki şekilde oluşturulur: <br/>
birincisi driver programda bulunan colloection' u ***parallelizing*** ederek. <br/>
ikincisi external storage system' da bulunan bir dataset' e referans vererek. external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.


### [Parallelized Collections](https://spark.apache.org/docs/latest/rdd-programming-guide.html#parallelized-collections)

Bir parallelized collection, sizin driver programınızda bulunan collection' ı parametre olarak olan ***SparkContext***' in ***parallelize*** metodu ile oluşturulur.

In [1]:
val data = Array(1, 2, 3, 4, 5)
val distData = sc.parallelize(data) //RDD oluştu ve paralel işlem yapılabilir artık.
// Normally, Spark tries to set the number of partitions automatically based on your cluster. 
// However, you can also set it manually by passing it as a second parameter to parallelize 

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.152:4040
SparkContext available as 'sc' (version = 3.2.0, master = local[*], app id = local-1643827632440)
SparkSession available as 'spark'


data: Array[Int] = Array(1, 2, 3, 4, 5)
distData: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:25


Parallel collection' larda önemli bir konuda bu RDD' nin kaç tane partition' a bölüneceği. Spark normalde partion' lama işlemlerini kendi yaparmış. Ama elle biz de verebilirmişiz. Elle vermek istersek `sc.parallelize(data, 10)` bu şekilde yapabiliriz.

> Bazı yerlerde `partions` yerine `slices` ifadesi geçebilirmiş.

## [External Datasets](https://spark.apache.org/docs/latest/rdd-programming-guide.html#external-datasets)

Spark can create distributed datasets from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra, HBase, Amazon S3, etc. Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.

Text file RDDs can be created using ***SparkContext***’s ***textFile*** method. This method takes a URI for the file (either a local path on the machine, or a hdfs://, s3a://, etc URI) and reads it as a collection of lines. Here is an example invocation:

In [2]:
import org.apache.spark.rdd.RDD;

val distFile: RDD[String] = sc.textFile("../README.md") //distFile' in türünü atamasını Scala' nın bulması yerine
// kendim atadığım için compile edilirken daha hızlı çalışacaktır.

import org.apache.spark.rdd.RDD
distFİle: org.apache.spark.rdd.RDD[String] = ../README.md MapPartitionsRDD[1] at textFile at <console>:27


Some notes on reading files with Spark:
- Bir tane ana bilgisayar var ve ona bağlı köle bilgisayarlar (worker node) var. Ana bilgisayarın local' ındeki file' a köle bilgisayarlar tabiki de erişemeyecek. Erişebilmesi için ya dosyayayı workerlara kopyala ya da internet üzerinden erişmelerini sağla
- All of Spark’s file-based input methods, including textFile, support running on directories, compressed files, and wildcards as well. For example, you can use textFile("/my/directory"), textFile("/my/directory/*.txt"), and textFile("/my/directory/*.gz").
- `textFile` metoduna isteğe bağlı RDD' nin partition' ını belirlemek için ikinci bir parametre daha verebiliyorsun. Default olarak bu partition' lara bölme işlemini `Spark zaten dosyadaki her bir block için bir tane partition oluşturarak yapıyor (HDFS' te her bir block 128MB' tır)`. İşte textFile metodunun ikinci parametresi sayesinde bu sayıyı arttırabiliyoruz. block değerinden aşağı bir partition sayısı belirleyemiyoruz. Default block size: 128MB. block size 256MB yapılamaz. `Default partition değeri 5 ise bu değeri parametre ile 4 yapamazsın. Çünkü partiton size' ı 128MB' den büyük olur.`

> Devamıda vardı yazmadım. İlgili olan başlığa tıklayarak gerekli linke gidebilir.

## [RDD Operations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations)

RDD' ler iki tür işleme destek verirler:
- **transformation**: var olan bir RDD' den yeni bir RDD elde etme.
- **actions**: Bir RDD' den belli hesaplamalar ile driver program' a bir değer döndürülmesi işlemleri.

Transformation işlemleri ile oluşan RDD' ler hemen oluşturulmazlarmış(transformation işlemleri saklanırmış). Eğer bir action gelirse RDD oluşturulur ve daha sonra action işlemi gerçekleştirilirmiş. Bu Spark' a verimlilik katarmış (Ben ikna olmadım).

Default olarak her action işlemi ile RDD oluşur ve üzerinden işlem yapılırmış ama istersek oluşan RDD' yi ***persist*** hale getirebiliyormuşuz: `action` ya da `persist` metodları ile. Eğer RDD üzerinde çok işlem yapılacaksa persist hale getirmek verimliliği arttır. (Buna ikna oldum)

### [Basic](https://spark.apache.org/docs/latest/rdd-programming-guide.html#basics)
> Spark' ın doc' u hakkikaten güzel anlatmış.

To illustrate RDD basics, consider the simple program below:
```scala
val lines = sc.textFile("data.txt")
val lineLengths = lines.map(s => s.length)
val totalLength = lineLengths.reduce((a, b) => a + b)
```
The first line defines a base RDD from an external file. This dataset is not loaded in memory or otherwise acted on: lines is merely a pointer to the file. The second line defines lineLengths as the result of a map transformation. Again, lineLengths is not immediately computed, due to laziness. Finally, we run reduce, which is an action. At this point Spark breaks the computation into tasks to run on separate machines, and each machine runs both its part of the map and a local reduction, returning only its answer to the driver program.

If we also wanted to use lineLengths again later, we could add:

```scala
lineLengths.persist()
```
before the reduce, which would cause lineLengths to be saved in memory after the first time it is computed.



### [Basic](https://spark.apache.org/docs/latest/rdd-programming-guide.html#basics)